In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import numpy as np

In [166]:
# Load data
path = '/home/han/repos/BEACO2N-CO2-Correction/data/'
X_train  = torch.Tensor(np.load(path + 'X_train.npy', allow_pickle=True))
X_test   = torch.Tensor(np.load(path + 'X_test.npy', allow_pickle=True))
X_val    = torch.Tensor(np.load(path + 'X_val.npy', allow_pickle=True))
y_train  = torch.Tensor(np.load(path + 'y_train.npy', allow_pickle=True))
y_test   = torch.Tensor(np.load(path + 'y_test.npy', allow_pickle=True))
y_val    = torch.Tensor(np.load(path + 't_val.npy', allow_pickle=True))

In [28]:
# Create sample check data
size = 1000
n_size = 2000

X_train = torch.randn(n_size, size)*32
y_train = X_train[:,30]**3 #+ X_train[:,2]/65 - 6*X_train[:,674]**2
print(y_train.shape)
#y_train = torch.exp(yy.reshape(-1,1))
tt = torch.randn(n_size,1) 
print(tt.shape)
y_train = y_train.reshape(-1,1) + tt
print(y_train.shape)

torch.Size([2000])
torch.Size([2000, 1])
torch.Size([2000, 1])


In [173]:
class co2Dataset(Dataset):
    def __init__(self, x_tensor, y_tensor):
        self.x = x_tensor
        self.y = y_tensor
        
    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.x)

train_data = co2Dataset(X_train,y_train)
train_loader = DataLoader(dataset=train_data, batch_size=30)

In [175]:
# Parameters
if torch.cuda.is_available():
    print("Cuda!")
    device   = 'cuda' 
else:
    print("Cpu :( ")
    device   = 'cpu'
    
n_iters      = 50
nsize_in     = X_train.shape[1] 
nsize_out    = 1
nsize_hidden1 = int(X_train.shape[1]/4)
nsize_hidden2 = int(nsize_hidden1/4)
nsize_hidden3 = int(nsize_hidden2/4)
n_layers     = 1
activations = [nn.Sigmoid(),#nn.ReLU(),
               nn.Sigmoid(),#nn.ReLU(),
               nn.Sigmoid(),#nn.ReLU(), 
               None]
layer_combiners = [nn.Linear(nsize_in,nsize_out,bias=False)]# nsize_hidden1),
                   #nn.Linear(nsize_hidden1, nsize_hidden2),
                   #nn.Linear(nsize_hidden2, nsize_hidden3),
                   #nn.Linear(nsize_hidden3, nsize_out)]
learning_rate = 0.0005
momentum = .5

Cuda :( 


In [129]:
def create_model(n_layers, activation, combiners):
    
    if len(activation) != n_layers:
        act = [activation[0]]*(n_layers - 1)
        act.append(activation[-1])
    else:
        act = activations
    if len(combiners) != n_layers:
        combine = [combiners[0]]*(n_layers - 1)
        combine.append(combiners[-1])
    else:
        combine = combiners
        
    for layer in range(n_layers):
        if layer == 0:
            model = nn.Sequential(combine[layer])
            if act[layer]:
                model.add_module(name=str(2*layer+1), module=act[layer])
        else:
            model.add_module(name=str(2*layer),module=combine[layer])
            if act[layer]:
                model.add_module(name=str(2*layer+1), module=act[layer])
                
    return model

In [134]:
def make_model_train_step( model, loss_fn, optimizer):
    
    def model_step(X, y):
        model.train()
        # Forward Pass
        y_pred = model(X)

        # Compute loss
        loss  = loss_fn(y, y_pred)
        print('epoch: ',epoch, ' loss: ', loss.item())
        # Perform a backward pass (backpropagation)
        loss.backward()

        # Update the parameters
        optimizer.step() 
        
        #Zero the gradients (so that gradients don't accumulate from epoch to epoch)
        optimizer.zero_grad()

        return loss.item()
    
    return model_step

In [184]:
model1 = create_model(n_layers, activations, layer_combiners)
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model1.parameters(),lr=0.000001, momentum=0, nesterov=False)

In [185]:
model_step = make_model_train_step(model1, loss_fn, optimizer)
losses = []

for epoch in range(100):
    
    for x_batch, y_batch in train_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        
        loss = model_step(x_batch, y_batch)
        #if np.isnan(loss):
        #    print("Loss has become not a number. I don't think we can recover from this. Stopping training loop.")
        #    break
        
        losses.append(loss)


epoch:  0  loss:  181634.484375
epoch:  0  loss:  75547.6953125
epoch:  0  loss:  31142.51171875
epoch:  0  loss:  13074.34765625
epoch:  0  loss:  5017.05322265625
epoch:  0  loss:  2762.916015625
epoch:  0  loss:  1332.6671142578125
epoch:  0  loss:  537.594970703125
epoch:  0  loss:  1045.2099609375
epoch:  0  loss:  1111.1632080078125
epoch:  0  loss:  696.048828125
epoch:  0  loss:  495.8585510253906
epoch:  0  loss:  1134.6337890625
epoch:  0  loss:  1348.6033935546875
epoch:  0  loss:  1193.493896484375
epoch:  0  loss:  536.48193359375
epoch:  0  loss:  885.4262084960938
epoch:  0  loss:  1054.6019287109375
epoch:  0  loss:  813.2329711914062
epoch:  0  loss:  1876.04248046875
epoch:  0  loss:  1868.4459228515625
epoch:  0  loss:  501.620361328125
epoch:  0  loss:  1321.285888671875
epoch:  0  loss:  709.3583374023438
epoch:  0  loss:  1875.634033203125
epoch:  0  loss:  1014.7990112304688
epoch:  0  loss:  386.1617431640625
epoch:  0  loss:  631.742919921875
epoch:  0  loss:  

epoch:  0  loss:  462.03277587890625
epoch:  0  loss:  852.282470703125
epoch:  0  loss:  1344.426513671875
epoch:  0  loss:  736.6734008789062
epoch:  0  loss:  306.8431701660156
epoch:  0  loss:  1051.1680908203125
epoch:  0  loss:  796.6578979492188
epoch:  0  loss:  692.5856323242188
epoch:  0  loss:  497.16558837890625
epoch:  0  loss:  1282.732666015625
epoch:  0  loss:  947.00146484375
epoch:  0  loss:  761.0648803710938
epoch:  0  loss:  841.3140258789062
epoch:  0  loss:  619.2001342773438
epoch:  0  loss:  834.78173828125
epoch:  0  loss:  1381.0479736328125
epoch:  0  loss:  1241.6907958984375
epoch:  0  loss:  585.1653442382812
epoch:  0  loss:  481.4284973144531
epoch:  0  loss:  1142.36767578125
epoch:  0  loss:  956.0189819335938
epoch:  0  loss:  514.590087890625
epoch:  0  loss:  842.0621948242188
epoch:  0  loss:  1299.583984375
epoch:  0  loss:  545.7415771484375
epoch:  0  loss:  1114.1051025390625
epoch:  0  loss:  1339.121826171875
epoch:  0  loss:  1061.097534179

epoch:  0  loss:  379.236572265625
epoch:  0  loss:  648.378173828125
epoch:  0  loss:  2293.560791015625
epoch:  0  loss:  1000.8187255859375
epoch:  0  loss:  422.8457336425781
epoch:  0  loss:  1303.0260009765625
epoch:  0  loss:  952.9523315429688
epoch:  0  loss:  1957.6136474609375
epoch:  0  loss:  1566.2930908203125
epoch:  0  loss:  1304.7337646484375
epoch:  0  loss:  767.1861572265625
epoch:  0  loss:  646.1389770507812
epoch:  0  loss:  1143.6268310546875
epoch:  0  loss:  524.5186157226562
epoch:  0  loss:  486.5916442871094
epoch:  0  loss:  1178.5074462890625
epoch:  0  loss:  957.0581665039062
epoch:  0  loss:  1010.2990112304688
epoch:  0  loss:  1112.1627197265625
epoch:  0  loss:  537.4783935546875
epoch:  0  loss:  1288.736328125
epoch:  0  loss:  1205.368896484375
epoch:  0  loss:  1200.5081787109375
epoch:  0  loss:  943.2900390625
epoch:  0  loss:  1246.0030517578125
epoch:  0  loss:  877.373291015625
epoch:  0  loss:  822.1128540039062
epoch:  0  loss:  876.1710

epoch:  1  loss:  1241.69580078125
epoch:  1  loss:  1235.3929443359375
epoch:  1  loss:  680.4747314453125
epoch:  1  loss:  468.5965576171875
epoch:  1  loss:  68618.7265625
epoch:  1  loss:  2402.7763671875
epoch:  1  loss:  1638.63232421875
epoch:  1  loss:  1501.4727783203125
epoch:  1  loss:  1131.3233642578125
epoch:  1  loss:  1374.7406005859375
epoch:  1  loss:  238.41061401367188
epoch:  1  loss:  1149.9964599609375
epoch:  1  loss:  744.7040405273438
epoch:  1  loss:  319.42864990234375
epoch:  1  loss:  2070.163818359375
epoch:  1  loss:  1166.02880859375
epoch:  1  loss:  1033.82080078125
epoch:  1  loss:  1568.499755859375
epoch:  1  loss:  1148.7655029296875
epoch:  1  loss:  1029.1671142578125
epoch:  1  loss:  506.10528564453125
epoch:  1  loss:  624.301513671875
epoch:  1  loss:  635.5440673828125
epoch:  1  loss:  1141.3524169921875
epoch:  1  loss:  1473.8045654296875
epoch:  1  loss:  738.75244140625
epoch:  1  loss:  425.75494384765625
epoch:  1  loss:  1159.50866

epoch:  1  loss:  980.5046997070312
epoch:  1  loss:  643.101806640625
epoch:  1  loss:  748.9728393554688
epoch:  1  loss:  1396.4356689453125
epoch:  1  loss:  1011.631591796875
epoch:  1  loss:  672.9169921875
epoch:  1  loss:  2062.706787109375
epoch:  1  loss:  1354.4931640625
epoch:  1  loss:  884.734375
epoch:  1  loss:  1512.4508056640625
epoch:  1  loss:  994.8215942382812
epoch:  1  loss:  901.2711791992188
epoch:  1  loss:  616.4708251953125
epoch:  1  loss:  672.3950805664062
epoch:  1  loss:  1159.7713623046875
epoch:  1  loss:  707.2918701171875
epoch:  1  loss:  643.3867797851562
epoch:  1  loss:  912.594970703125
epoch:  1  loss:  578.3729248046875
epoch:  1  loss:  1147.8516845703125
epoch:  1  loss:  1417.580322265625
epoch:  1  loss:  1055.0479736328125
epoch:  1  loss:  404.9002990722656
epoch:  1  loss:  1741.1287841796875
epoch:  1  loss:  1015.5233154296875
epoch:  1  loss:  1537.0633544921875
epoch:  1  loss:  941.85546875
epoch:  1  loss:  780.0867919921875
epo

...

epoch:  97  loss:  339.5908203125
epoch:  97  loss:  1757.984130859375
epoch:  97  loss:  486.4712829589844
epoch:  97  loss:  442.78228759765625
epoch:  97  loss:  1578.6551513671875
epoch:  97  loss:  325.0962829589844
epoch:  97  loss:  487.79302978515625
epoch:  97  loss:  744.2449340820312
epoch:  98  loss:  1088.427001953125
epoch:  98  loss:  1212.3455810546875
epoch:  98  loss:  697.7510375976562
epoch:  98  loss:  1183.1400146484375
epoch:  98  loss:  579.9762573242188
epoch:  98  loss:  422.4925231933594
epoch:  98  loss:  414.7109069824219
epoch:  98  loss:  212.0260772705078
epoch:  98  loss:  840.5316772460938
epoch:  98  loss:  906.8458251953125
epoch:  98  loss:  567.6372680664062
epoch:  98  loss:  446.3984375
epoch:  98  loss:  888.4454956054688
epoch:  98  loss:  973.714111328125
epoch:  98  loss:  1022.9374389648438
epoch:  98  loss:  566.5548095703125
epoch:  98  loss:  759.3480834960938
epoch:  98  loss:  1058.0076904296875
epoch:  98  loss:  758.7481079101562
epoc

epoch:  98  loss:  1029.7410888671875
epoch:  98  loss:  939.3658447265625
epoch:  98  loss:  454.6439208984375
epoch:  98  loss:  800.4209594726562
epoch:  98  loss:  1021.2928466796875
epoch:  98  loss:  701.2828369140625
epoch:  98  loss:  305.7257385253906
epoch:  98  loss:  797.0211181640625
epoch:  98  loss:  655.4702758789062
epoch:  98  loss:  589.2365112304688
epoch:  98  loss:  495.0085144042969
epoch:  98  loss:  1065.5316162109375
epoch:  98  loss:  846.6255493164062
epoch:  98  loss:  624.2778930664062
epoch:  98  loss:  778.9453735351562
epoch:  98  loss:  477.1244812011719
epoch:  98  loss:  668.0944213867188
epoch:  98  loss:  1044.0028076171875
epoch:  98  loss:  714.9879150390625
epoch:  98  loss:  527.1432495117188
epoch:  98  loss:  456.0234680175781
epoch:  98  loss:  721.804931640625
epoch:  98  loss:  752.4663696289062
epoch:  98  loss:  340.6041564941406
epoch:  98  loss:  621.8876953125
epoch:  98  loss:  1121.6962890625
epoch:  98  loss:  415.4198913574219
epo

epoch:  98  loss:  1078.833251953125
epoch:  98  loss:  699.2136840820312
epoch:  98  loss:  1752.01806640625
epoch:  98  loss:  1548.876953125
epoch:  98  loss:  1255.46875
epoch:  98  loss:  579.9810791015625
epoch:  98  loss:  462.5226745605469
epoch:  98  loss:  886.0991821289062
epoch:  98  loss:  467.800048828125
epoch:  98  loss:  374.7620544433594
epoch:  98  loss:  1006.415283203125
epoch:  98  loss:  921.5971069335938
epoch:  98  loss:  885.4476318359375
epoch:  98  loss:  750.6782836914062
epoch:  98  loss:  504.3669738769531
epoch:  98  loss:  934.7562255859375
epoch:  98  loss:  1195.0389404296875
epoch:  98  loss:  1044.5782470703125
epoch:  98  loss:  886.7544555664062
epoch:  98  loss:  1157.308837890625
epoch:  98  loss:  794.949462890625
epoch:  98  loss:  682.5663452148438
epoch:  98  loss:  802.814697265625
epoch:  98  loss:  1280.530029296875
epoch:  98  loss:  1206.4229736328125
epoch:  98  loss:  762.306396484375
epoch:  98  loss:  664.0026245117188
epoch:  98  l

epoch:  99  loss:  981.5489501953125
epoch:  99  loss:  716.2083740234375
epoch:  99  loss:  304.493408203125
epoch:  99  loss:  1529.5714111328125
epoch:  99  loss:  870.6041259765625
epoch:  99  loss:  1066.3238525390625
epoch:  99  loss:  1448.724853515625
epoch:  99  loss:  1044.0955810546875
epoch:  99  loss:  794.0253295898438
epoch:  99  loss:  464.8412170410156
epoch:  99  loss:  550.2831420898438
epoch:  99  loss:  592.5635986328125
epoch:  99  loss:  927.7881469726562
epoch:  99  loss:  1237.0362548828125
epoch:  99  loss:  602.9107055664062
epoch:  99  loss:  275.348876953125
epoch:  99  loss:  968.8624267578125
epoch:  99  loss:  955.7821655273438
epoch:  99  loss:  446.9029846191406
epoch:  99  loss:  1321.0926513671875
epoch:  99  loss:  888.869384765625
epoch:  99  loss:  392.999267578125
epoch:  99  loss:  934.7166748046875
epoch:  99  loss:  359.9767150878906
epoch:  99  loss:  1378.176025390625
epoch:  99  loss:  1108.6646728515625
epoch:  99  loss:  882.987548828125


epoch:  99  loss:  2031.253173828125
epoch:  99  loss:  602.3171997070312
epoch:  99  loss:  2186.851318359375
epoch:  99  loss:  424.17144775390625
epoch:  99  loss:  558.0381469726562
epoch:  99  loss:  878.5231323242188
epoch:  99  loss:  570.0150756835938
epoch:  99  loss:  654.2869262695312
epoch:  99  loss:  1228.0380859375
epoch:  99  loss:  984.4005126953125
epoch:  99  loss:  768.2819213867188
epoch:  99  loss:  1354.8760986328125
epoch:  99  loss:  1217.483154296875
epoch:  99  loss:  891.6824340820312
epoch:  99  loss:  1367.1578369140625
epoch:  99  loss:  809.884033203125
epoch:  99  loss:  746.323486328125
epoch:  99  loss:  412.5814514160156
epoch:  99  loss:  556.3897094726562
epoch:  99  loss:  1127.7213134765625
epoch:  99  loss:  644.2499389648438
epoch:  99  loss:  584.9502563476562
epoch:  99  loss:  787.754150390625
epoch:  99  loss:  578.783935546875
epoch:  99  loss:  931.5685424804688
epoch:  99  loss:  940.8250122070312
epoch:  99  loss:  920.7202758789062
epo

In [183]:
# Reset the model
m = model1
for name, module in m.named_children():
    print('resetting ', name)
    module.reset_parameters()

resetting  0


In [123]:
# Lets Evaluate
model.eval()
print(model.training ) #Just to be sure

y_pred = model(X_test)
loss  = criterion(y_pred, y_test)
print("Loss: ", loss)

NameError: name 'model' is not defined

In [36]:
ypred = model1(X_train)
err = torch.norm(ypred-y_train, p=2)
print(torch.mean(y_train))
print(ypred[1:10])
print(err)

tensor(-10.7292)
tensor([[-768.],
        [-768.],
        [-768.],
        [-768.],
        [-768.],
        [-768.],
        [-768.],
        [-768.],
        [-768.]], grad_fn=<SliceBackward>)
tensor(5711471., grad_fn=<NormBackward0>)


In [45]:
a = (torch.std(X_train,0) >= 1.25)
torch.sum(a)


tensor(143)

In [180]:
torch.__version__

'1.6.0'

In [90]:
optimizer.state_dict()

{'state': {},
 'param_groups': [{'lr': 0.01,
   'momentum': 0.9,
   'dampening': 0,
   'weight_decay': 0,
   'nesterov': True,
   'params': [0, 1]}]}

In [48]:
loss_fn._state_dict_hooks

NameError: name 'loss_fn' is not defined

In [181]:
torch.utils.collect_env

AttributeError: module 'torch.utils' has no attribute 'collect_env'